In [0]:
import pandas as pd
import numpy as np
import pickle
import sys
import os
import io
import re
from sys import path
import numpy as np
import pickle
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from string import punctuation, digits
from IPython.core.display import display, HTML
from nltk.corpus import stopwords
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

In [18]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [19]:
import os
os.chdir("/gdrive/My Drive/")
os.getcwd()

'/gdrive/My Drive'

In [20]:
! pip install googletrans

In [21]:
!pip install emoji

In [45]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
# google translation
from googletrans import Translator
# from google.cloud import translate
import emoji,json,copy
def translate_to_eng(data):
    translator = Translator()
    for i in range(1,data_size):
            inpt = emoji.demojize(data[i])
            print(lang[i],str(inpt))
            translation = translator.translate(str(inpt),dest='en')
            print(translation.text)
            data[i] = str(translation.text)
    return data

In [0]:
# training data1
fp = open("./preprocessed-twitter-tweets/processedPositive.txt")
data = fp.read()
data_pos = data.split(',')
label_pos = np.array(np.ones(len(data_pos)))
fp = open("./preprocessed-twitter-tweets/processedNegative.txt")
data = fp.read()
data_neg = data.split(',')
label_neg = np.array(2 * np.ones(len(data_neg)))
fp = open("./preprocessed-twitter-tweets/processedNeutral.txt")
data = fp.read()
data_neu = data.split(',')
label_neu = np.array(np.zeros(len(data_neu)))

for i in range(0,len(data_pos)):
    data_pos[i] = data_pos[i] +" $$ 1" 
for i in range(0,len(data_neg)):
    data_neg[i] = data_neg[i] +" $$ 2" 
for i in range(0,len(data_neu)):
    data_neu[i] = data_neu[i] +" $$ 0" 

In [0]:
#training data 2
fp = open("gold-post-eng.txt")
gold_data = fp.read().split('\n')

In [28]:
data_all = data_pos + data_neg + data_neu + gold_data
print(len(data_all))

12927


In [33]:
import random
random.shuffle(data_all)
data = []
labels = []
count = 0
for i in range(0,len(data_all)):
    if len(data_all[i].split('$$')) == 2:
        try:
            labels.append(int(data_all[i].split('$$')[1]))
            data.append(data_all[i].split('$$')[0])
        except:
            count +=1

print(len(data),len(labels))

11665 11665


In [0]:
import re
def preprocessing(eng_data):
    for i in range(0,len(eng_data)):
        eng_data[i] = eng_data[i].lower()
        eng_data[i] = re.sub('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+', '',eng_data[i])
        eng_data[i] = re.sub('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}', '',eng_data[i])
        eng_data[i] = re.sub('[^\w\s]','',eng_data[i])
        eng_data[i] = re.sub('\d', '',eng_data[i])
        word_tokens = word_tokenize(eng_data[i])
        filtered_sentence = [w for w in word_tokens if not w in stopwords.words('english')]
        sent = ""
        i = 0
        for w in filtered_sentence:
            if i ==0:
                sent += w
                i+=1
            else:
                sent += " "+w
        eng_data[i] = sent
    return eng_data

In [35]:
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=0.10, random_state=0)
classes = np.array([0,1,2])

print(len(X_train),len(X_test),len(y_train),len(y_test))

10498 1167 10498 1167


In [0]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier

In [37]:
vectorizer = TfidfVectorizer(analyzer='word',ngram_range=(1,2), max_features=50000,max_df=0.5,use_idf=True, norm='l2') 
counts = vectorizer.fit_transform(X_train)
vocab = vectorizer.vocabulary_
classifier = SGDClassifier(alpha=1e-05,max_iter=50,penalty='elasticnet')
targets = y_train
classifier = classifier.fit(counts, targets)
example_counts = vectorizer.transform(X_test)
predictions = classifier.predict(example_counts)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


In [0]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report

In [39]:
acc = accuracy_score(y_test, predictions, normalize=True)
hit = precision_score(y_test, predictions, average=None,labels=classes)
capture = recall_score(y_test, predictions, average=None,labels=classes)
print('Model Accuracy:%.2f'%acc)
print(classification_report(y_test, predictions))

Model Accuracy:0.76
              precision    recall  f1-score   support

           0       0.78      0.81      0.79       582
           1       0.77      0.79      0.78       329
           2       0.68      0.61      0.64       256

   micro avg       0.76      0.76      0.76      1167
   macro avg       0.74      0.73      0.74      1167
weighted avg       0.75      0.76      0.75      1167



In [0]:
# model saving
import pickle
filename = 'Model_nlp_hackathon.sav'
pickle.dump(classifier, open(filename, 'wb'))

In [0]:
#testing Model
# loading test data 
import xlrd
wb = xlrd.open_workbook('./Languages_Sentiment_labels2.xlsx')
sheet = wb.sheet_by_index(0)
data_test = []
labels_code_test = []
labels_test =[]
lang_test = []
data_size = 413
for i in range(1,data_size):
    labels_test.append(sheet.cell_value(i,0))
    lang_test.append(sheet.cell_value(i,1))
    data_test.append(sheet.cell_value(i,2)) 

n=0
p=0
neg=0
data_size = len(data_test)
for i in range(0,len(data_test)):
    if labels_test[i] == "neutral":
        labels_code_test.append(0)
        n+=1
    elif labels_test[i] == "positive":
        labels_code_test.append(1)
        p+=1
    else:
        labels_code_test.append(2)
        neg+=1
        
data_test = translate_to_eng(data_test)


In [56]:
example_counts = vectorizer.transform(preprocessing(data_test))
predictions = classifier.predict(example_counts)
acc = accuracy_score(labels_code_test, predictions, normalize=True)
hit = precision_score(labels_code_test, predictions, average=None,labels=classes)
capture = recall_score(labels_code_test, predictions, average=None,labels=classes)
print('Model Accuracy:%.2f'%acc)
print(classification_report(labels_code_test, predictions))

Model Accuracy:0.44
              precision    recall  f1-score   support

           0       0.54      0.53      0.54       176
           1       0.37      0.61      0.46       132
           2       0.43      0.09      0.14       104

   micro avg       0.44      0.44      0.44       412
   macro avg       0.45      0.41      0.38       412
weighted avg       0.46      0.44      0.41       412



In [59]:
print("Language","text","True_sentiment","predicted_sentiment")
labels_t = []
labels_p = []
for i in range(0,len(labels_code_test)):
  if labels_code_test[i]==0:
    labels_t.append("neutral")
  elif labels_code_test[i] == 1:
    labels_t.append("positive")
  else:
    labels_t.append("negative")
for i in range(0,len(predictions)):
  if predictions[i]==0:
    labels_p.append("neutral")
  elif predictions[i] == 1:
    labels_p.append("positive")
  else:
    labels_p.append("negative")    
for i in range(0,len(labels_code_test)):
  print(lang_test[i],labels_t[i], labels_p[i])

Language text True_sentiment predicted_sentiment
bn positive positive
bn neutral positive
bn negative neutral
bn neutral neutral
bn negative neutral
bn neutral neutral
bn neutral neutral
bn neutral neutral
bn positive neutral
bn neutral neutral
bn negative neutral
bn negative neutral
bn negative neutral
bn neutral neutral
bn positive positive
bn positive neutral
bn neutral neutral
bn negative neutral
bn negative neutral
bn positive neutral
bn neutral neutral
bn positive positive
bn negative neutral
bn neutral neutral
bn positive positive
bn neutral neutral
bn positive neutral
bn positive neutral
bn positive positive
bn positive neutral
bn positive neutral
bn negative neutral
bn neutral neutral
bn positive neutral
bn positive neutral
bn negative neutral
bn positive neutral
bn neutral neutral
bn positive neutral
bn neutral neutral
bn neutral neutral
bn positive neutral
bn positive positive
bn positive neutral
bn positive neutral
bn neutral neutral
bn neutral positive
bn neutral neutral
b